# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import googlemaps
from datetime import datetime
import citipy
from citipy import citipy

# Import API key
from api_keys import gkey

gmaps.configure(api_key=gkey)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read in the data

city_data_df = pd.read_csv("../output_data/my_cities.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gogrial,8.5332,28.1004,29.72,60,99,1.56,SS,1627566763
1,1,cape town,-33.9258,18.4232,15.06,90,75,0.45,ZA,1627566765
2,2,vallenar,-28.5708,-70.7581,18.65,22,0,0.64,CL,1627566767
3,3,padang,-0.9492,100.3543,25.03,84,94,0.93,ID,1627566000
4,4,pevek,69.7008,170.3133,0.30,97,100,12.04,RU,1627566771
...,...,...,...,...,...,...,...,...,...,...
559,559,mazabuka,-15.8560,27.7480,23.15,25,48,4.33,ZM,1627567940
560,560,istisu,40.9457,48.0738,22.11,59,46,2.09,AZ,1627567942
561,561,winnemucca,40.9730,-117.7357,17.03,34,1,0.00,US,1627567944
562,562,havre-saint-pierre,50.2334,-63.5986,11.55,83,100,3.04,CA,1627567946


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create a heat map that displays the humidity for every city from the part I of the homework.

# Store 'Lat' and 'Lng' into  locations
coordinates = city_data_df[['Lat', 'Lng']].astype(float)
coordinates

,Lat,Lng
0,8.5332,28.1004
1,-33.9258,18.4232
2,-28.5708,-70.7581
3,-0.9492,100.3543
4,69.7008,170.3133
...,...,...
559,-15.8560,27.7480
560,40.9457,48.0738
561,40.9730,-117.7357
562,50.2334,-63.5986


In [4]:
# Create a heat map
fig = gmaps.figure(zoom_level = 1.9, center = (33.7,00))

heat_layer = gmaps.heatmap_layer(coordinates, weights=city_data_df['Humidity'], 
                                 dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
    # A max temperature lower than 25 celsius degrees but higher than 20 celsius.
    # Wind speed less than 10 mph.
    # Zero cloudiness.
    # Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
city_data_filtered_df = city_data_df.loc[city_data_df['Max Temp'] < 25]
city_data_filtered_df = city_data_filtered_df.loc[city_data_filtered_df['Max Temp'] > 20]
city_data_filtered_df = city_data_filtered_df.loc[city_data_filtered_df['Cloudiness'] == 0]
city_data_filtered_df = city_data_filtered_df.loc[city_data_filtered_df['Wind Speed'] < 10]
city_data_filtered_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
145,145,guerrero negro,27.9769,-114.0611,21.12,86,0,4.46,MX,1627567065
186,186,richards bay,-28.7830,32.0377,21.83,36,0,4.34,ZA,1627567151
202,202,huambo,-12.7761,15.7392,24.89,23,0,5.14,AO,1627567182
215,215,aranos,-24.1333,19.1167,22.41,14,0,3.79,NaN,1627567214
227,227,alta floresta,-9.8756,-56.0861,23.52,30,0,3.78,BR,1627567207
272,272,karratha,-20.7377,116.8463,21.84,74,0,3.82,AU,1627567346
281,281,luderitz,-26.6481,15.1594,20.44,41,0,9.58,NaN,1627567367
313,313,kemi,65.8596,24.8228,22.87,60,0,4.12,FI,1627567430
316,316,piet retief,-27.0071,30.8132,20.63,14,0,3.18,ZA,1627567440
354,354,pemba,-12.9740,40.5178,24.89,59,0,6.88,MZ,1627567520


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create the Hotel DataFrame

hotel_df = pd.DataFrame()

# add the columns needed
hotel_df['City'] = city_data_filtered_df['City']
hotel_df['Country'] = city_data_filtered_df['Country']

# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# partial url
# url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=photos,formatted_address,name,opening_hours,rating&locationbias=circle:2000@37.4222339,-122.0854804"
url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=photos,formatted_address,name,opening_hours,rating&locationbias=circle:"
units = "metric"
dist = 5000
#lat = -18.06
#lng = 13.84

# combinne them into a single dataframe
city_data_filtered_df['Locations'] = list(zip(city_data_filtered_df['Lat'], city_data_filtered_df['Lng']))

cities = []
hotel = []
address = []

# Loop through the list of locations to get the nearby hotels
for location in city_data_filtered_df['Locations']:
    lat, lng = location
    city = citipy.nearest_city(lat, lng).city_name
    if city not in cities:
        cities.append(city)    
        # Build query URL
        query_url = f"{url}{dist}@{lat},{lng}&key={gkey}"
        response = requests.get(query_url).json()
        hotel.append(response['candidates'][0]['name'])
        address. append(response['candidates'][0]['formatted_address'])
              
hotel_df['Hotel Name'] = hotel
hotel_df['Address'] = address
hotel_df['Lat'] = city_data_filtered_df['Lat']
hotel_df['Lng'] = city_data_filtered_df['Lng']

hotel_df


,City,Country,Hotel Name,Address,Lat,Lng
145,guerrero negro,MX,The Halfway Inn,"Paralelo 28, México 1, 23940 Guerrero Negro, B...",27.9769,-114.0611
186,richards bay,ZA,BON Hotel Waterfront Richards Bay,"Corner of Bridgetown and Pioneer Road, TuziGaz...",-28.7830,32.0377
202,huambo,AO,Hotel Ekuikui I,"Av. Torres Garcia, Huambo, Angola",-12.7761,15.7392
215,aranos,NaN,Aranos Kalahariland Guest Farm,"Aranos, Namibia",-24.1333,19.1167
227,alta floresta,BR,Hotel Caleche Park Hotel,"Av. Perimetral Rogério Silva, 1850 - Centro, A...",-9.8756,-56.0861
272,karratha,AU,Karratha International Hotel,"Corner of Hillview and Millstream Roads, Karra...",-20.7377,116.8463
281,luderitz,NaN,LÜDERITZ NEST HOTEL,"820 Dias St, Luderitz, Namibia",-26.6481,15.1594
313,kemi,FI,Seaside Glass Villas,"Lumilinnankatu 15, 94100 Kemi, Finland",65.8596,24.8228
316,piet retief,ZA,Dusk to Dawn Guest House,"Farm Wagendrift, Piet Retief, 2380, South Africa",-27.0071,30.8132
354,pemba,MZ,Avani Pemba Beach Hotel,"Avenida da Marginal, Cabo Delgado Pemba, 5470,...",-12.9740,40.5178


In [7]:
# Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

fig = gmaps.figure(zoom_level = 1.9, center = (33.7,00))

heat_layer = gmaps.heatmap_layer(coordinates, weights=city_data_df['Humidity'], 
                                 dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer on top of heat map


# Display figure
